https://www.kaggle.com/datasets/lnasiri007/ieeecis-fraud-detection 

In [2]:
import pandas as pd
import numpy as np

tx = pd.read_csv(r"C:\Users\louis\Downloads\R&D PROJECTS\data\train_transaction.csv")

# merge identity features 
try:
    iden = pd.read_csv(r"C:\Users\louis\Downloads\R&D PROJECTS\data\train_identity.csv")
    df_ieee = tx.merge(iden, on="TransactionID", how="left")
except:
    df_ieee = tx.copy()

df_ieee.shape, df_ieee.head()

((590540, 434),
    TransactionID  isFraud  TransactionDT  TransactionAmt ProductCD  card1  \
 0        2987000        0          86400            68.5         W  13926   
 1        2987001        0          86401            29.0         W   2755   
 2        2987002        0          86469            59.0         W   4663   
 3        2987003        0          86499            50.0         W  18132   
 4        2987004        0          86506            50.0         H   4497   
 
    card2  card3       card4  card5  ...                id_31  id_32  \
 0    NaN  150.0    discover  142.0  ...                  NaN    NaN   
 1  404.0  150.0  mastercard  102.0  ...                  NaN    NaN   
 2  490.0  150.0        visa  166.0  ...                  NaN    NaN   
 3  567.0  150.0  mastercard  117.0  ...                  NaN    NaN   
 4  514.0  150.0  mastercard  102.0  ...  samsung browser 6.2   32.0   
 
        id_33           id_34  id_35 id_36 id_37  id_38  DeviceType  \
 0       

In [3]:
# drop the columns with too many missing values

# Separate label
y = df_ieee["isFraud"]

# Drop ID and label
X = df_ieee.drop(columns=["isFraud", "TransactionID"])

# Keep only numeric columns (simpler + safer)
X = X.select_dtypes(include=["int64", "float64"])

# Drop columns with >50% missing values
missing_ratio = X.isnull().mean()
X = X.loc[:, missing_ratio < 0.5]

X.shape

(590540, 209)

In [4]:
# Train test split

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [5]:
# Handle missing values

from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

model = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("clf", RandomForestClassifier(
        n_estimators=100,
        random_state=42,
        n_jobs=-1,
        class_weight="balanced_subsample"
    ))
])

model.fit(X_train, y_train)

Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('clf',
                 RandomForestClassifier(class_weight='balanced_subsample',
                                        n_jobs=-1, random_state=42))])

In [6]:
# evaluation - classification report ROC-AUC

from sklearn.metrics import classification_report, roc_auc_score

y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99    113975
           1       0.92      0.43      0.58      4133

    accuracy                           0.98    118108
   macro avg       0.95      0.71      0.79    118108
weighted avg       0.98      0.98      0.97    118108

ROC-AUC: 0.922412917032045


The model achieved 92% precision, 43% recall, and ROC-AUC of 0.92, reflecting good discrimination ability but reduced fraud recall in a realistic, high-dimensional dataset.

With recall being at a low of 43%, with the default classification threshold (0.5), many fraud cases are not being flagged.